### Installing the important libraries

In [1]:
"""
conda install -c conda-forge swig -y
conda install -c conda-forge box2d-py -y
python -m pip install --upgrade pip setuptools wheel
python -m pip install gymnasium[box2d]
"""

'\nconda install -c conda-forge swig -y\nconda install -c conda-forge box2d-py -y\npython -m pip install --upgrade pip setuptools wheel\npython -m pip install gymnasium[box2d]\n'

### Importing the important modules

It is preferred to train the model on a device with a GPU

In [2]:
import gymnasium as gym
import matplotlib
from matplotlib.animation import FuncAnimation
from torch.distributions import MultivariateNormal
from torch.distributions import Categorical
import matplotlib.pyplot as plt
import torch
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import namedtuple, deque
from itertools import count
import random
import math
import pickle
from torch.optim import Adam
import time
import os
import glob
from datetime import datetime


is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display
plt.ion()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Implementation of the Car Racing game from Gymnasium using Proximal Policy Optimization

https://gymnasium.farama.org/environments/box2d/car_racing/#


## What is Proximal Policy Optimization (PPO)?

**Proximal Policy Optimization (PPO)** is a Reinforcement Learning (RL) algorithm used to train *agents* to perform tasks. Developed by OpenAI in 2017, it has become an industry standard because it strikes an excellent balance between:

- Ease of implementation  
- Sample efficiency  
- Stability and ease of tuning  

In the referenced notebook, PPO is used to train a car to drive in the `CarRacing-v3` environment.

---

## How PPO Works: Core Concepts

PPO belongs to the family of **Policy Gradient** methods. Instead of learning a value table or Q-function, it directly learns a **policy**—a mapping from observations to actions.

### 1. Actor–Critic Architecture

The implementation uses two neural networks:

- **Actor**  
  - Takes the observation (processed pixels)
  - Outputs a probability distribution over actions
  - Responsible for *acting*

- **Critic**  
  - Takes the same observation
  - Predicts the expected future reward (value)
  - Helps evaluate how good the Actor’s actions were

The Critic provides a learning signal to stabilize and guide the Actor.

---

### 2. The “Proximal” Idea: Clipped Objective Function

A major challenge in policy gradient methods is that large updates can completely destabilize learning.

PPO solves this using a **clipped surrogate objective**.

#### Probability Ratio

During training, PPO computes the ratio:

```
ratio = π_new(a|s) / π_old(a|s)
```

- `ratio = 1` → no change  
- `ratio >> 1 or << 1` → update is too aggressive  

#### Clipping

The ratio is clipped to a fixed interval, typically:

```
[1 − ε, 1 + ε]
```

This prevents excessively large policy updates, ensuring **stable and monotonic improvement**.

---

### 3. Generalized Advantage Estimation (GAE)

The notebook uses a discounted return computation with a `lambda_` parameter. This corresponds to **Generalized Advantage Estimation (GAE)**.

GAE balances:

- **Low variance** (bootstrapping from the Critic)
- **Low bias** (using actual rewards)

It estimates the **advantage**:

```
Advantage = How much better an action was than expected
```

This improves learning stability and efficiency.

---

## Breakdown of the Implementation

| Feature | Implementation Detail |
|------|------------------------|
| Observation | Frames resized to 84×84, grayscale, normalized |
| Frame stacking | 4 frames stacked to capture motion |
| Action space | Discrete actions (`action_dim = 5`) |
| Exploration | `Categorical` action distribution |
| Clipping | `self.clip = 0.4` |
| Value loss | Mean Squared Error (MSE) |
| Policy loss | PPO clipped objective |

---

## Training Loop Logic

1. **Data Collection**  
   - The agent interacts with the environment
   - Observations, actions, rewards, and log-probabilities are stored

2. **Advantage Computation**  
   - The Critic estimates state values
   - GAE computes advantages

3. **Policy Optimization**  
   - Actor is updated to increase probability of good actions
   - Clipping prevents large destructive updates

4. **Value Function Update**  
   - Critic is trained to better predict returns

---

## Why PPO Works Well

- Stable training
- Few hyperparameters
- Works well with high-dimensional inputs (images)
- Robust to imperfect tuning

These properties make PPO a strong choice for environments like **CarRacing-v2**.

---


## 1. Setup and Preprocessing

### Observation Space:
- A top-down 96x96 RGB image of the car and race track.

To make learning efficient, we can't just throw raw RGB frames at the network. The following section handles State Representation:

- **Preprocessing:** We convert images to grayscale and resize them to $84 \times 84$ to reduce the computational load.
- **Frame Skipping:** The agent doesn't need to see every single frame to understand movement. We skip frames to speed up training.
- **Frame Stacking:** A single static image doesn't show velocity. By stacking the 4 most recent frames, we provide the network with temporal context (motion).

In [3]:
def image_preprocessing(img):
  img = cv2.resize(img, dsize=(84, 84))
  img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) / 255.0
  return img

Set up the environment wrapper

In [4]:
class CarEnvironment(gym.Wrapper):
  def __init__(self, env, skip_frames=4, stack_frames=4, no_operation=50, **kwargs):
    super().__init__(env, **kwargs)
    self._no_operation = no_operation
    self._skip_frames = skip_frames
    self._stack_frames = stack_frames

  def reset(self):
    observation, info = self.env.reset()

    for i in range(self._no_operation):
      observation, reward, terminated, truncated, info = self.env.step(0)

    observation = image_preprocessing(observation)
    self.stack_state = np.tile(observation, (self._stack_frames, 1, 1))
    return self.stack_state, info


  def step(self, action):
    total_reward = 0
    for i in range(self._skip_frames):
      observation, reward, terminated, truncated, info = self.env.step(action)
      total_reward += reward

      if terminated or truncated:
        break

    observation = image_preprocessing(observation)
    self.stack_state = np.concatenate((self.stack_state[1:], observation[np.newaxis]), axis=0)
    return self.stack_state, total_reward, terminated, truncated, info


In [5]:
class Actor(nn.Module):
  def __init__(self, in_channels, out_channels, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self._n_features = 32 * 9 * 9

    self.conv = nn.Sequential(
        nn.Conv2d(in_channels, 16, kernel_size=8, stride=4),
        nn.ReLU(),
        nn.Conv2d(16, 32, kernel_size=4, stride=2),
        nn.ReLU(),
    )

    self.fc = nn.Sequential(
        nn.Linear(self._n_features, 256),
        nn.ReLU(),
        nn.Linear(256, out_channels),
    )


  def forward(self, x):
    x = self.conv(x)
    x = x.view((-1, self._n_features))
    x = self.fc(x)
    return x



class Critic(nn.Module):
  def __init__(self, in_channels, out_channels, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self._n_features = 32 * 9 * 9

    self.conv = nn.Sequential(
        nn.Conv2d(in_channels, 16, kernel_size=8, stride=4),
        nn.ReLU(),
        nn.Conv2d(16, 32, kernel_size=4, stride=2),
        nn.ReLU(),
    )

    self.fc = nn.Sequential(
        nn.Linear(self._n_features, 256),
        nn.ReLU(),
        nn.Linear(256, out_channels),
    )


  def forward(self, x):
    x = self.conv(x)
    x = x.view((-1, self._n_features))
    x = self.fc(x)
    return x

In [6]:
class PPO:
  def __init__(self, action_dim=5, obs_dim=4, episodes=1500, trajectories=300, gamma=0.99, lr_actor=0.0001, lr_critic=0.0001, clip=0.4, n_updates=3, lambda_=0.99):
    self.action_dim = action_dim
    self.obs_dim = obs_dim
    self.episodes = episodes
    self.trajectories = trajectories
    self.gamma = gamma
    self.lr_actor = lr_actor
    self.lr_critic = lr_critic
    self.clip = clip
    self.n_updates = n_updates
    self.lambda_ = lambda_
    self._total_rewards = []
    self.actor = Actor(obs_dim, action_dim).to(device)
    self.critic = Critic(obs_dim, 1).to(device)
    self.actor_optim = Adam(self.actor.parameters(), lr=self.lr_actor)
    self.critic_optim = Adam(self.critic.parameters(), lr=self.lr_critic)



  """
  This function takes as a parameter an observation, feeds it to the CNN and gets the
  raw predictions (logits) and it samples an action through the categorical distribution.
  It returns the action and the logarithmic probability.
  """
  def get_action(self, obs):
    # Our observation is a 2D numpy array so we first create a tensor
    obs = torch.tensor(obs, dtype=torch.float32, device=device).unsqueeze(0)

    # Feeding the tensor to the actor and get the logits
    action_probs = self.actor(obs)

    # Creating a Categorical distribution
    dist = Categorical(logits=action_probs)

    # Sampling the action
    action = dist.sample()

    log_prob = dist.log_prob(action)

    return action.detach().cpu().numpy(), log_prob.detach()


  """
  This function is where we collect the trajectories (e.g. observations, rewards and other information)
  using the current policy. We run this until we collect the number of trajectories we set.
  It returns all the collected information.
  """
  def collect_trajectories(self):
    batch_obs = []
    batch_rewards = []
    batch_log_probs = []
    batch_next_obs = []
    batch_actions = []
    batch_dones = []
    t = 0

    # Creating the discrete environment and passing it through the our wrapper for the modification
    env = gym.make('CarRacing-v3', continuous=False, render_mode='rgb_array')
    env = CarEnvironment(env)

    while True:

      # Reset environment
      obs, _ = env.reset()

      # Runs as many times as needed until we get the number of trajectories we want
      while True:

        # Append current state
        batch_obs.append(obs)

        # Choose an action
        a, log_prob = self.get_action(obs)

        # Append action
        batch_actions.append(a)

        # Append log prob
        batch_log_probs.append(log_prob)

        # Perform the action
        obs, rew, terminated, truncated, _ = env.step(a.item())

        # Append reward
        batch_rewards.append(rew)

        # Increase the number of T horizon
        t += 1

        # Check criterion for loop termination
        if terminated or truncated or t == self.trajectories:
          batch_dones.append(1)
          break
        else:
          batch_dones.append(0)

      # Check criterion for loop termination
      if t == self.trajectories:
        env.close()
        break

    self._total_rewards.append(sum(batch_rewards))

    # Convert to tensors
    batch_obs = np.array(batch_obs)
    batch_obs = torch.tensor(batch_obs, dtype=torch.float32)
    batch_rewards = torch.tensor(batch_rewards, dtype=torch.float32)
    batch_log_probs = torch.tensor(batch_log_probs, dtype=torch.float32)
    batch_actions = torch.tensor(batch_actions, dtype=torch.long)

    # Reward Normalization
    batch_rewards = (batch_rewards - batch_rewards.mean()) / (batch_rewards.std() + 1e-8)

    return batch_obs, batch_rewards, batch_log_probs, batch_actions, batch_dones


  """
  Computing the discounted reward sum based on the current V values with
  GAE (Generalized Advantage Estimation)
  """
  def compute_discounted_sum(self, batch_rewards, V, batch_dones):
    discounted_sum = []
    gae = 0
    zero = torch.tensor([0])
    V = torch.cat((V.cpu(), zero))

    for i in reversed(range(len(batch_rewards))):
      delta = batch_rewards[i] + self.gamma * V[i + 1] * (1 - batch_dones[i]) - V[i]
      gae = delta + self.gamma * self.lambda_ * gae * (1 - batch_dones[i])
      discounted_sum.insert(0, gae)

    return discounted_sum


  """
  Make the agent learn the environment
  """
  def train(self):

    # Running the training phase for some episodes
    for episode in range(self.episodes):

      if episode % 10 == 0:
        print(self._total_rewards)

      if (1 + episode) % 50 == 0:
        print("Processed: ", episode + 1)
        print()
        torch.save(model.actor.state_dict(), f'actor_weights_{episode + 1}.pth')
        torch.save(model.critic.state_dict(), f'critic_weights_{episode + 1}.pth')
        with open('statistics.pkl', 'wb') as f:
          pickle.dump((self._total_rewards), f)

      # Collecting the batches with the information
      batch_obs, batch_rewards, batch_log_probs, batch_actions, batch_dones = self.collect_trajectories()

      # Compute V values with the critic network in current states
      V = self.critic(batch_obs.to(device)).squeeze()

      # Compute the discounted sum
      discounted_sum = self.compute_discounted_sum(batch_rewards, V, batch_dones)
      discounted_sum = torch.tensor(discounted_sum, dtype=torch.float32)

      # The advantages to maximize
      advantages = discounted_sum - V.detach().cpu()
      advantages = advantages.clone()

      # Normalizing the advantage sum to help the network from exploding gradients
      advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

      # Update the network
      for update in range(self.n_updates):

        # Finding the probabilites of our collected observations
        actions_probs = self.actor(batch_obs.to(device))

        # Gathering the probabilities of the sampled actions (current policy through categorical sampling)
        action_log_probs = actions_probs.gather(1, batch_actions.to(device)).squeeze()

        # Computing the probability ratio between the new and old policy
        ratios = torch.exp(action_log_probs - batch_log_probs.to(device)).cpu()

        # Computing the normal policy gradient objective
        surr1 = ratios * advantages

        # Clipping the policy ratio
        surr2 = torch.clamp(ratios, 1 - self.clip, 1 + self.clip) * advantages

        # Picking the minimum of the two surrogate objects and adding minus so we can create a minimization problem
        loss = -torch.min(surr1, surr2).mean()

        # Back propagation, returning the losses for both actor & critic networks
        self.actor_optim.zero_grad()
        loss.backward(retain_graph=True)
        self.actor_optim.step()

        V = self.critic(batch_obs.to(device)).squeeze()
        value_loss = nn.MSELoss()(V, discounted_sum.detach().to(device))

        self.critic_optim.zero_grad()
        value_loss.backward()
        self.critic_optim.step()


In [ ]:
model = PPO()
model.train()

[]


/opt/miniconda3/envs/rl_gymnasium/lib/python3.11/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists
/tmp/ipykernel_3487809/2009179959.py:110: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch_actions = torch.tensor(batch_actions, dtype=torch.long)


[-47.24769354938889, -50.826775210578404, -39.63215547703245, -41.738986063934576, -76.53374613003139, -59.74112736698834, -31.534880136292188, -52.10745906610115, -37.719728660259875, -50.243036282452366]
[-47.24769354938889, -50.826775210578404, -39.63215547703245, -41.738986063934576, -76.53374613003139, -59.74112736698834, -31.534880136292188, -52.10745906610115, -37.719728660259875, -50.243036282452366, -43.538031771836366, -22.770550576184363, -23.032258136201577, -65.17491132290876, 12.645520581113015, -61.02279495990902, -38.00756288522448, -61.428649635036884, -30.389308104411384, -26.525646879756472]
[-47.24769354938889, -50.826775210578404, -39.63215547703245, -41.738986063934576, -76.53374613003139, -59.74112736698834, -31.534880136292188, -52.10745906610115, -37.719728660259875, -50.243036282452366, -43.538031771836366, -22.770550576184363, -23.032258136201577, -65.17491132290876, 12.645520581113015, -61.02279495990902, -38.00756288522448, -61.428649635036884, -30.38930810

In [ ]:
eval_env = gym.make('CarRacing-v3', continuous=False, render_mode='rgb_array')
eval_env = CarEnvironment(eval_env)

frames = []
scores = 0
s, _ = eval_env.reset()

done, ret = False, 0

while not done:
    frames.append(eval_env.render())
    s = torch.tensor(s, dtype=torch.float32, device=device).unsqueeze(0)
    a = torch.argmax(model.actor(s), dim=-1)
    discrete_action = a.item() % 5
    s_prime, r, terminated, truncated, info = eval_env.step(discrete_action)
    s = s_prime
    ret += r
    done = terminated or truncated
    if terminated:
      print(terminated)
scores += ret

In [ ]:
def animate(imgs, video_name, _return=True):
    import cv2
    import os
    import string
    import random

    if video_name is None:
        video_name = ''.join(random.choice(string.ascii_letters) for i in range(18)) + '.webm'
    height, width, layers = imgs[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'VP90')
    video = cv2.VideoWriter(video_name, fourcc, 10, (width, height))

    for img in imgs:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        video.write(img)
    video.release()

animate(frames, None)